In [2]:
import pandas as pd
import numpy as np
# np.set_printoptions(threshold=np.nan)
import json
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder

/Users/Ayush/condaenv/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
data = pd.read_csv('lora_GTW.csv', index_col=0)
# data.columns = pd.MultiIndex.from_tuples(data.columns.str.split("|").tolist())

In [4]:
data.head()

,gateway_id,pkt_date_and_time,pkt_rssi,pkt_snr,pkt_data,pkt_size,pkt_longitude,pkt_latitude
id,,,,,,,,
1,240AC4F01E023E3C,2018-02-06T15:14:31.677078Z,-51,6,"{""time"": ""2018-02-06T15:14:31.536828Z"", ""gtiD""...",67,4.58,0.55
2,240AC4F01E0286DC,2018-02-06T15:14:31.673436Z,-44,7,"{""time"": ""2018-02-06T15:14:31.536828Z"", ""gtiD""...",67,4.58,0.55
3,240AC4F01E025FF4,2018-02-06T15:14:34.308128Z,-44,7,"{""time"": ""2018-02-06T15:14:34.171153Z"", ""gtiD""...",67,1.50,9.05
4,240AC4F01E023E3C,2018-02-06T15:14:34.311056Z,-45,7,"{""time"": ""2018-02-06T15:14:34.171153Z"", ""gtiD""...",67,1.50,9.05
5,240AC4F01E023C54,2018-02-06T15:14:34.312091Z,-40,7,"{""time"": ""2018-02-06T15:14:34.171153Z"", ""gtiD""...",67,1.50,9.05


### Pre-processing

##### Separate pkt_data into two different columns -> pkt_rec_time, pkt_rec_from

In [5]:
time = pd.Series()
gtiD = pd.Series()

for e in data.pkt_data:
    entry = json.loads(e)
    time = time.append(pd.Series(entry['time']), ignore_index=True)
    gtiD = gtiD.append(pd.Series(entry['gtiD']), ignore_index=True)

In [6]:
data = data.assign(pkt_rec_time=time)
data = data.assign(pkt_rec_from=gtiD)
data = data.drop(['pkt_data'], axis=1)

In [7]:
data.head()

,gateway_id,pkt_date_and_time,pkt_rssi,pkt_snr,pkt_size,pkt_longitude,pkt_latitude,pkt_rec_time,pkt_rec_from
id,,,,,,,,,
1,240AC4F01E023E3C,2018-02-06T15:14:31.677078Z,-51,6,67,4.58,0.55,2018-02-06T15:14:31.536828Z,240AC4F01E025FF4
2,240AC4F01E0286DC,2018-02-06T15:14:31.673436Z,-44,7,67,4.58,0.55,2018-02-06T15:14:34.171153Z,240AC4F01E0286DC
3,240AC4F01E025FF4,2018-02-06T15:14:34.308128Z,-44,7,67,1.50,9.05,2018-02-06T15:14:34.171153Z,240AC4F01E0286DC
4,240AC4F01E023E3C,2018-02-06T15:14:34.311056Z,-45,7,67,1.50,9.05,2018-02-06T15:14:34.171153Z,240AC4F01E0286DC
5,240AC4F01E023C54,2018-02-06T15:14:34.312091Z,-40,7,67,1.50,9.05,2018-02-06T15:14:34.595861Z,240AC4F01E023C54


In [8]:
set(data.gateway_id.values) | set(data.pkt_rec_from.values)

{'240AC4F01E023C54',
 '240AC4F01E023E3C',
 '240AC4F01E025FF4',
 '240AC4F01E0286DC',
 nan}

###### Encoding of gateway_id values

In [9]:
gateways = set(data.gateway_id.values) | set(data.pkt_rec_from.values)
# gateways = {
#     "240AC4F01E025FF4": 0,
#     "240AC4F01E0286DC": 1,
#     "240AC4F01E023E3C": 2,
#     "240AC4F01E023C54": 3
# }
num_gateways = len(gateways)

In [10]:
# Label Encoder
le = LabelEncoder()
le.fit(list(gateways));

In [11]:
def convert_gtiD_to_num(column):
    encoded_col = pd.Series()
    for entry in column:
        en = le.transform([str(entry)])[0]
        encoded_col = encoded_col.append(pd.Series(en), ignore_index=True)
    return encoded_col

In [12]:
data = data.assign(gateway_id=convert_gtiD_to_num(data.gateway_id))
data = data.assign(pkt_rec_from=convert_gtiD_to_num(data.pkt_rec_from))

In [13]:
data.head()

,gateway_id,pkt_date_and_time,pkt_rssi,pkt_snr,pkt_size,pkt_longitude,pkt_latitude,pkt_rec_time,pkt_rec_from
id,,,,,,,,,
1,3.0,2018-02-06T15:14:31.677078Z,-51,6,67,4.58,0.55,2018-02-06T15:14:31.536828Z,3.0
2,2.0,2018-02-06T15:14:31.673436Z,-44,7,67,4.58,0.55,2018-02-06T15:14:34.171153Z,3.0
3,1.0,2018-02-06T15:14:34.308128Z,-44,7,67,1.50,9.05,2018-02-06T15:14:34.171153Z,3.0
4,0.0,2018-02-06T15:14:34.311056Z,-45,7,67,1.50,9.05,2018-02-06T15:14:34.171153Z,0.0
5,3.0,2018-02-06T15:14:34.312091Z,-40,7,67,1.50,9.05,2018-02-06T15:14:34.595861Z,0.0


In [14]:
def one_hot_encoder(gateway):
    """
        gateway: str
        returns one-hot encoded array
    """
    entry = [0] * num_gateways
    entry[gateways[gateway]] = 1
    return entry

##### Create training + test set

In [15]:
# X = np.array(data[['gateway_id', 'pkt_rssi', 'pkt_snr', 'pkt_data']])
X = np.array(data[['gateway_id', 'pkt_rssi', 'pkt_snr', 'pkt_rec_from', 'pkt_longitude', 'pkt_latitude']])
y = np.array(list(zip(data.pkt_longitude, data.pkt_latitude)))

In [16]:
train_ratio = 0.8
train_len = int(train_ratio * len(X))

In [17]:
X_train, X_test = X[:train_len], X[train_len + 1:]
y_train, y_test = y[:train_len], y[train_len + 1:]
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

### Model

In [18]:
input_dim = X.shape[1]

In [19]:
model = Sequential()
model.add(Dense(input_dim, input_dim=input_dim))
model.add(Dense(10, activation='relu'))
# model.add(Dense, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [20]:
model.get_config()

[{'class_name': 'Dense',
  'config': {'activation': 'linear',
   'activity_regularizer': None,
   'batch_input_shape': (None, 6),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'dtype': 'float32',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_1',
   'trainable': True,
   'units': 6,
   'use_bias': True}},
 {'class_name': 'Dense',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uni

In [21]:
model.fit(X_train, y_train, epochs=5,  batch_size=1);

Epoch 1/5
2602/2602 [==============================]2602/2602 [==============================] - 4s 1ms/step - loss: 10261560105.9914 - acc: 0.7498

Epoch 2/5
2602/2602 [==============================]2602/2602 [==============================] - 3s 1ms/step - loss: 10.7512 - acc: 0.7494

Epoch 3/5
2602/2602 [==============================]2602/2602 [==============================] - 3s 1ms/step - loss: 10.7379 - acc: 0.7494

Epoch 4/5
2602/2602 [==============================]2602/2602 [==============================] - 3s 1ms/step - loss: 10.7536 - acc: 0.7494

Epoch 5/5
2602/2602 [==============================]2602/2602 [==============================] - 3s 1ms/step - loss: 10.7154 - acc: 0.7494



In [22]:
loss_and_metrics = model.evaluate(X_test, y_test);

650/650 [==============================]650/650 [==============================] - 0s 75us/step



In [23]:
prediction = model.predict(X_test)

In [24]:
loss_and_metrics

[nan, 0.7507692304024329]

In [25]:
prediction

array([[2.8380716, 5.1412044],
       [2.8380716, 5.1412044],
       [2.8380716, 5.1412044],
       ...,
       [2.8380716, 5.1412044],
       [2.8380716, 5.1412044],
       [      nan,       nan]], dtype=float32)

In [26]:
y_test

array([[1.5 , 9.05],
       [4.58, 9.05],
       [4.58, 9.05],
       ...,
       [0.33, 0.66],
       [4.58, 0.55],
       [4.58, 0.55]])